# Histogram of Oriented Gradients Search #

These functions extract HOG features from an image then sub-sample to get the overlaying windows. 

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from plot_images import *

Helper functions for feature extraction

In [2]:
def convertColorSpace(img, colorIn):
    if colorIn == 'RGB':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if colorIn == 'BGR':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    
def extractHog(img, orient=9, pix_per_cell=8, n_cell_per_block=2, 
               feature_vec=False, vis=False):
    # Call with two outputs if an image of the HOG is requested
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(n_cell_per_block, n_cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(n_cell_per_block, n_cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def binColorsSpatially(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
def computeColorHistogram(img, n_bins=32):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=n_bins)
    channel2_hist = np.histogram(img[:,:,1], bins=n_bins)
    channel3_hist = np.histogram(img[:,:,2], bins=n_bins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

def scaleFeatures(features):
    X = np.stack(features, axis=0)
    scaler = StandardScaler().fit(X)
    return scaler.transform(X)

The function that can extract features using hog sub-sampling

In [3]:
def findCars(img, color_space_in, ystart, ystop, scale, svc, 
             hog_orient, hog_pix_per_cell, hog_cells_per_block, 
             spatial_size, hist_bins):
    
    imgDisplay = np.copy(img)
    img = img.astype(np.float32)/255
    
    imgRoi = convertColorSpace(img[ystart:ystop,:,:], color_space_in)
    if scale != 1:
        imshape = imgRoi.shape
        imgRoi = cv2.resize(imgRoi, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = imgRoi[:,:,0]
    ch2 = imgRoi[:,:,1]
    ch3 = imgRoi[:,:,2]

    # Define blocks and steps as above
    n_xBlocks = (ch1.shape[1] // hog_pix_per_cell) - hog_cells_per_block + 1
    n_yBlocks = (ch1.shape[0] // hog_pix_per_cell) - hog_cells_per_block + 1 
    n_featuresPerBlock = n_orientationBins*hog_cells_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    n_winBlocks = (window // hog_pix_per_cell) - hog_cells_per_block + 1
    n_cellsPerStep = 2  # Instead of overlap, define how many cells to step
    n_xSteps = (n_xBlocks - n_winBlocks) // n_cellsPerStep
    n_ySteps = (n_yBlocks - n_winBlocks) // n_cellsPerStep
    
    # Compute individual channel HOG features for the entire image
    hog1 = extractHog(ch1, hog_orient, hog_pix_per_cell, hog_cells_per_block)
    hog2 = extractHog(ch2, hog_orient, hog_pix_per_cell, hog_cells_per_block)
    hog3 = extractHog(ch3, hog_orient, hog_pix_per_cell, hog_cells_per_block)
    
    for xb in range(n_xSteps):
        for yb in range(n_ySteps):
            ypos = yb*n_cellsPerStep
            xpos = xb*n_cellsPerStep
            # Extract HOG for this patch
            hogFeat1 = hog1[ypos:ypos+n_winBlocks, xpos:xpos+n_winBlocks].ravel() 
            hogFeat2 = hog2[ypos:ypos+n_winBlocks, xpos:xpos+n_winBlocks].ravel() 
            hogFeat3 = hog3[ypos:ypos+n_winBlocks, xpos:xpos+n_winBlocks].ravel() 
            hogFeatures = np.hstack((hogFeat1, hogFeat2, hogFeat3))

            xleft = xpos*hog_pix_per_cell
            ytop = ypos*hog_pix_per_cell

            # Extract the image patch
            imgSub = cv2.resize(imgRoi[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            binColorFeatures = binColorsSpatially(imgSub, spatial_size)
            histFeatures = computeColorHistogram(imgSub, hist_bins)

            # Scale features and make a prediction
            testFeatures = scaleFeatures([binColorFeatures, histFeatures, hogFeatures])
            testPrediction = svc.predict(test_features)
            
            if testPrediction == 1:
                xLeft = np.int(xleft*scale)
                yTop = np.int(ytop*scale)
                frame = np.int(window*scale)
                cv2.rectangle(imgDisplay,(xLeft, yTop+ystart),
                              (xLeft+frame,yTop+frame+ystart),(0,0,255),6) 
                
    return imgDisplay

In [5]:
def getTestingImages(pathname):
    files = glob.glob(pathname)
    images = []
    for fname in files:
        img = mpimg.imread(fname)
        images.append(img)
    return images

testImages = getTestingImages('test_images/test*.jpg')  
testTitles = []
testCmap = []
for i in range(0, len(testImages)):
    testTitles.append("Test #{}".format(i))
    testCmap.append(None)
   

color_space_in = 'RGB'
ystart = 400
ystop = 656
scale = 1.5
hog_orient = 9 
hog_pix_per_cell = 8 
hog_cells_per_block = 2 
spatial_size = 8
hist_bins = 40
    
testOutputs = []
for img in testImages:
    iOut = findCars(img, color_space_in, ystart, ystop, svc, 
                    hog_orient, hog_pix_per_cell, hog_cells_per_block, 
                    spatial_size, hist_bins)
    testOutputs.append(iOut)

plotMany(2, 3, testOutputs, testTitles, testCmap)


NameError: name 'svc' is not defined